In [0]:
from pyspark.sql.functions import lit, to_timestamp, date_format, current_timestamp, current_date, expr, col
import requests
import pandas as pd
import json
import os
import json

### ## # Consulta de API ->https://www.alphavantage.co

In [0]:
API_KEY = os.environ.get('APY_KEY_COTACAO')

In [0]:
symbol = 'NVDA'
url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&outputsize=full&apikey={API_KEY}'
r = requests.get(url)
data = r.json()
print(json.dumps(data, indent=4, ensure_ascii=False))
# displayHTML(f"<pre>{json.dumps(data, indent=4, ensure_ascii=False)}</pre>")

In [0]:
serie_diaria = data.get('Time Series (Daily)', {})
if serie_diaria:
    ultima_data = sorted(serie_diaria.keys())[-1]
    fechamento = float(serie_diaria[ultima_data]['4. close'])

    print(f'Último fechamento ({ultima_data}): ${fechamento:.2f}')
else:
    print('Não foi possível obter a série diária')

In [0]:
df_ultimo = pd.DataFrame([{f'cotacao_{symbol}': fechamento, 'ultima_data': ultima_data,}])
df_cotacao = spark.createDataFrame(df_ultimo).withColumn('data_ingestion_bronze', lit(current_timestamp()))

In [0]:
# 42:44

In [0]:
(
    df_cotacao
    .write
    .mode("append")
    .format("delta")
    .saveAsTable("monitoramento_dados.bronze.cotacao_nvidia")
)
